<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Naas - Emailbuilder demo
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Emailbuilder_demo.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Naas+-+Emailbuilder+demo:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #naas #emailbuilder #snippet #operations

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

**Description:** This notebook provides a demonstration of the Naas Emailbuilder, a tool for creating and managing email campaigns.

## Input

### Import libraries

In [ ]:
from naas_drivers import emailbuilder
import naas
import os
from naas_data_product import *
from naas_drivers import naasauth

### Setup Variables
- `linkedin_url`: This variable is used for storing the URL of the LinkedIn profile to be updated. It is retrieved from an environment variable called "LINKEDIN_PROFILE_URL" using the os.environ.get() method. If the environment variable is not set, the naas.secret.get() method is used to retrieve the URL from a secret store.
- `output_dir`: This variable is used for storing the path to the directory where the output files of the LinkedIn profile update process will be saved. In this case, the path is set to "/home/ftp/content-engine-linkedin-profile/outputs/linkedin/profile".
- `subdir`: This variable is used for storing the name of a subdirectory where the output files will be saved. In this case, the subdirectory is set to "12_mr_month".
- `graphs`: This variable is used for storing a list of graph types that will be updated for the LinkedIn profile. The available graph types are "published", "views", "likes", "comments", and "shares".
- `email_to`: This variable is used for storing a list of email addresses that will receive the notification email. In this case, the list contains the email address of the script user, which is retrieved using the naasauth.connect().user.me().get("username") method.
- `subject`: This variable is used for storing the subject of the notification email. In this case, the subject is "🖼️ Content Engine: LinkedIn profile update".
- `email_content`: This variable is used for storing the content of the notification email. In this case, it is an empty dictionary, which can be populated with additional content later.

In [ ]:
# Inputs
linkedin_url = os.environ.get("LINKEDIN_PROFILE_URL") or naas.secret.get("LINKEDIN_PROFILE_URL")
output_dir = os.path.join(OUTPUTS_PATH, "linkedin", "profile")
subdir = "12_mr_month"
graphs = ["published", "views", "likes", "comments", "shares"]

# Outputs
email_to = [naasauth.connect().user.me().get("username")] # List to emails address of the receiver(s)
subject = "🖼️ Content Engine: LinkedIn profile update" # Email subject
email_content = {}

## Model

### Constants

In [ ]:
NAAS_WEBSITE = "https://www.naas.ai/"
IMAGE_NAAS = "https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"

### Build the email

In [ ]:
# Create header naas
header_naas = (f"<a href='{NAAS_WEBSITE}'>"
               f"<img align='center' width='30%' target='_blank' style='border-radius:5px;'"
               f"src='{IMAGE_NAAS}'"
               "alt='Daily update to analyze and improve your professional presence on LinkedIn.'/>"
               "</a>")

# Update email content
email_content = {
    "title": header_naas,
    "heading": emailbuilder.heading("Content Engine - LinkedIn profile")
}

for graph in graphs:
    # Get profile ID
    profile_id = get_linkedin_id(linkedin_url)
    
    # Create output dir
    output_dir_path = os.path.join(output_dir, profile_id, "posts", graph, subdir, "png")

    # Get last file updated in dir
    output_path = get_last_updated_file(output_dir_path, file_type="png")
    
    # Check if asset exists
    asset_link = naas.asset.find(output_path)
    if not asset_link:
        # Add asset
        asset_link = naas.asset.add(output_path)
        
    # Add asset to email content dict
    email_content[graph] = emailbuilder.image(asset_link, align="center")
    email_content[f"{graph}_text"] = emailbuilder.text("")

# Add footer to email
email_content["footer_company"] = emailbuilder.footer_company(naas=True)

# Generate email
content = emailbuilder.generate(display="iframe", **email_content)

## Output

### Send the email

In [ ]:
naas.notification.send(
    email_to=email_to,
    subject=subject,
    html=content,
)